## Limpeza de dados Cartola ano 2016

In [34]:
from pyspark.sql import HiveContext
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import lower, col, lit, regexp_replace
from pyspark.sql.functions import lit
import pandas as pd
import json
import requests

## Jogadores

In [37]:
#Carrega arquivo CSV
jogadores_2016_df = spark.read.option("encoding", "UTF-8").csv("/cartola/data/2016/2016_jogadores.csv", header=True)
sorted_jogadores_2016_df = jogadores_2016_df.sort(jogadores_2016_df.ID.desc())


In [38]:
#Adiciona coluna ANO = 2016
#Clube ID está como NA????
jogadores_2016_ano = sorted_jogadores_2016_df.withColumn('ano', lit(2016))
jogadores_2016_ano.show()

+-----+-----------------+-------+---------+----+
|   ID|          Apelido|ClubeID|PosicaoID| ano|
+-----+-----------------+-------+---------+----+
|97460|      João Victor|    344|        4|2016|
|97451|   Talysson Lalau|    294|        4|2016|
|97450| Gustavo Mosquito|    294|        5|2016|
|97449|         Carvalho|    294|        4|2016|
|97448| Thalisson Kelven|    294|        3|2016|
|97445|Matheus Iacovelli|    275|        5|2016|
|97444|      Julio Rusch|    294|        4|2016|
|97443|         Romercio|    294|        3|2016|
|97442|        Léo Cotia|    344|        5|2016|
|97441|      Lukas Silva|    344|        1|2016|
|97438|           Lyncon|    303|        5|2016|
|97437|       Marquinhos|    303|        4|2016|
|97436|        Handerson|    303|        3|2016|
|97433|      Guilherme  |    327|        5|2016|
|97432|  Victor Henrique|    327|        3|2016|
|97431|            Diego|    327|        2|2016|
|97352|            Pachu|    263|        4|2016|
|97350|         Emil

In [41]:
jogadores_2016_ano.write.parquet('/cartola/clean/jogadores/2016')

In [42]:
#transforma arquivo em parquet
jogadores_2016_parquet = spark.read.parquet('/cartola/clean/jogadores/*')

In [44]:
jogadores_2016_parquet.show()
jogadores_2016_parquet.schema


+-----+--------------------+-------+---------+----+
|   ID|             Apelido|ClubeID|PosicaoID| ano|
+-----+--------------------+-------+---------+----+
|51683|        Bruno Rangel|    315|        5|2014|
|51705|       Bruno Rodrigo|    283|        3|2014|
|51772|     Éverton Ribeiro|    283|        4|2014|
|51779|       Pedro Botelho|    282|        2|2014|
|51781|               Ávine|    265|        2|2014|
|51985|      Anderson Pedra|    292|        4|2014|
|41387|       Eduardo Costa|    314|        4|2015|
|41327|Vanderlei Luxemburgo|    262|        6|2015|
|41258|     Adilson Batista|     NA|        6|2015|
|41218|              Magrão|    292|        1|2015|
|41159|         João Carlos|     NA|        3|2015|
|97460|         João Victor|    344|        4|2016|
|97451|      Talysson Lalau|    294|        4|2016|
|97450|    Gustavo Mosquito|    294|        5|2016|
|97449|            Carvalho|    294|        4|2016|
|97448|    Thalisson Kelven|    294|        3|2016|
|97445|   Ma

StructType(List(StructField(ID,StringType,true),StructField(Apelido,StringType,true),StructField(ClubeID,StringType,true),StructField(PosicaoID,StringType,true),StructField(ano,IntegerType,true)))

In [45]:
#Alterar ID para int.(está string)
jogadores_2016_parquet = jogadores_2016_parquet.withColumn("ID", jogadores_2016_parquet["ID"].cast(IntegerType()))
jogadores_2016_parquet = jogadores_2016_parquet.withColumn("ClubeID", jogadores_2016_parquet["ClubeID"].cast(IntegerType()))
jogadores_2016_parquet = jogadores_2016_parquet.withColumn("PosicaoID", jogadores_2016_parquet["PosicaoID"].cast(IntegerType()))

In [46]:
jogadores_2016_parquet.schema

StructType(List(StructField(ID,IntegerType,true),StructField(Apelido,StringType,true),StructField(ClubeID,IntegerType,true),StructField(PosicaoID,IntegerType,true),StructField(ano,IntegerType,true)))

In [47]:
jogadores_2016_parquet.show()

print(jogadores_2016_parquet.count())
#null é o Técnico

+-----+--------------------+-------+---------+----+
|   ID|             Apelido|ClubeID|PosicaoID| ano|
+-----+--------------------+-------+---------+----+
|51683|        Bruno Rangel|    315|        5|2014|
|51705|       Bruno Rodrigo|    283|        3|2014|
|51772|     Éverton Ribeiro|    283|        4|2014|
|51779|       Pedro Botelho|    282|        2|2014|
|51781|               Ávine|    265|        2|2014|
|51985|      Anderson Pedra|    292|        4|2014|
|41387|       Eduardo Costa|    314|        4|2015|
|41327|Vanderlei Luxemburgo|    262|        6|2015|
|41258|     Adilson Batista|   null|        6|2015|
|41218|              Magrão|    292|        1|2015|
|41159|         João Carlos|   null|        3|2015|
|97460|         João Victor|    344|        4|2016|
|97451|      Talysson Lalau|    294|        4|2016|
|97450|    Gustavo Mosquito|    294|        5|2016|
|97449|            Carvalho|    294|        4|2016|
|97448|    Thalisson Kelven|    294|        3|2016|
|97445|   Ma

## Partidas_IDS (temos partidas como NA)

In [60]:
partidas_ids_2016_df = spark.read.option("encoding", "UTF-8").csv("/cartola/data/2016/2016_partidas_ids.csv", header=True)

In [61]:
partidas_ids_2016_ano = partidas_ids_2016_df.withColumn('ano', lit(2016))

In [62]:
partidas_ids_2016_ano.show()

+------+------+------+-----------+----------+---------------+---------+----+
|    ID|Rodada|CasaID|VisitanteID|PlacarCasa|PlacarVisitante|Resultado| ano|
+------+------+------+-----------+----------+---------------+---------+----+
|201233|     1|   327|        266|         0|              1|Visitante|2016|
|201235|     1|   344|        287|         4|              1|     Casa|2016|
|201239|     2|   277|        294|         2|              1|     Casa|2016|
|201242|     2|   293|        282|         1|              1|   Empate|2016|
|201240|     2|   283|        316|         2|              2|   Empate|2016|
|201237|     2|   266|        344|         2|              2|   Empate|2016|
|201243|     2|   315|        327|         3|              1|     Casa|2016|
|201238|     2|   276|        285|         1|              2|Visitante|2016|
|201234|     1|   282|        277|         1|              0|     Casa|2016|
|201244|     2|   303|        275|         2|              1|     Casa|2016|

In [63]:
partidas_ids_2016_ano.write.parquet('/cartola/clean/partidas_ids/2016')

In [64]:
partidas_ids_2016_parquet = spark.read.parquet('/cartola/clean/partidas_ids/*')

In [65]:
partidas_ids_2016_parquet.show()
partidas_ids_2016_parquet.schema

+------+------+------+-----------+----------+---------------+---------+----+
|    ID|Rodada|CasaID|VisitanteID|PlacarCasa|PlacarVisitante|Resultado| ano|
+------+------+------+-----------+----------+---------------+---------+----+
|201233|     1|   327|        266|         0|              1|Visitante|2016|
|201235|     1|   344|        287|         4|              1|     Casa|2016|
|201239|     2|   277|        294|         2|              1|     Casa|2016|
|201242|     2|   293|        282|         1|              1|   Empate|2016|
|201240|     2|   283|        316|         2|              2|   Empate|2016|
|201237|     2|   266|        344|         2|              2|   Empate|2016|
|201243|     2|   315|        327|         3|              1|     Casa|2016|
|201238|     2|   276|        285|         1|              2|Visitante|2016|
|201234|     1|   282|        277|         1|              0|     Casa|2016|
|201244|     2|   303|        275|         2|              1|     Casa|2016|

StructType(List(StructField(ID,StringType,true),StructField(Rodada,StringType,true),StructField(CasaID,StringType,true),StructField(VisitanteID,StringType,true),StructField(PlacarCasa,StringType,true),StructField(PlacarVisitante,StringType,true),StructField(Resultado,StringType,true),StructField(ano,IntegerType,true)))

In [67]:
partidas_ids_2016_parquet = partidas_ids_2016_parquet.withColumn("ID", partidas_ids_2016_parquet["ID"].cast(IntegerType()))
partidas_ids_2016_parquet = partidas_ids_2016_parquet.withColumn("Rodada", partidas_ids_2016_parquet["Rodada"].cast(IntegerType()))
partidas_ids_2016_parquet = partidas_ids_2016_parquet.withColumn("CasaID", partidas_ids_2016_parquet["CasaID"].cast(IntegerType()))
partidas_ids_2016_parquet = partidas_ids_2016_parquet.withColumn("VisitanteID", partidas_ids_2016_parquet["VisitanteID"].cast(IntegerType()))
partidas_ids_2016_parquet = partidas_ids_2016_parquet.withColumn("PlacarCasa", partidas_ids_2016_parquet["PlacarCasa"].cast(IntegerType()))
partidas_ids_2016_parquet = partidas_ids_2016_parquet.withColumn("PlacarVisitante", partidas_ids_2016_parquet["PlacarVisitante"].cast(IntegerType()))

In [68]:
partidas_ids_2016_parquet.show()

print(partidas_ids_2016_parquet.count())

+------+------+------+-----------+----------+---------------+---------+----+
|    ID|Rodada|CasaID|VisitanteID|PlacarCasa|PlacarVisitante|Resultado| ano|
+------+------+------+-----------+----------+---------------+---------+----+
|201233|     1|   327|        266|         0|              1|Visitante|2016|
|201235|     1|   344|        287|         4|              1|     Casa|2016|
|201239|     2|   277|        294|         2|              1|     Casa|2016|
|201242|     2|   293|        282|         1|              1|   Empate|2016|
|201240|     2|   283|        316|         2|              2|   Empate|2016|
|201237|     2|   266|        344|         2|              2|   Empate|2016|
|201243|     2|   315|        327|         3|              1|     Casa|2016|
|201238|     2|   276|        285|         1|              2|Visitante|2016|
|201234|     1|   282|        277|         1|              0|     Casa|2016|
|201244|     2|   303|        275|         2|              1|     Casa|2016|

## scouts_raw

In [89]:
scouts_raw_2016_df = spark.read.option("encoding", "UTF-8").csv("/cartola/data/2016/2016_scouts_raw.csv", header=True)

In [90]:
scouts_raw_2016_ano = scouts_raw_2016_df.withColumn('ano', lit(2016))

In [91]:
scouts_raw_2016_ano.limit(10).toPandas()

,Rodada,ClubeID,AtletaID,Participou,Pontos,PontosMedia,Preco,PrecoVariacao,FS,PE,...,RB,FC,GC,CA,CV,SG,DD,DP,GS,ano
0,0,294,37623,FALSE,0,0,4,0,0,0,...,0,0,0,0,0,0,0,0,0,2016
1,1,294,37623,TRUE,5.3,5.3,6.84,2.84,0,3,...,1,1,0,0,0,1,0,0,0,2016
2,2,294,37623,TRUE,-0.5,2.4,6.67,-0.17,0,0,...,0,1,0,0,0,0,0,0,0,2016
3,3,294,37623,TRUE,2.3,2.37,6.98,0.31,1,2,...,2,2,0,0,0,0,0,0,0,2016
4,4,294,37623,TRUE,-2.7,1.1,5.5,-1.48,0,3,...,1,3,0,1,0,0,0,0,0,2016
5,5,294,37623,TRUE,8.2,2.52,7.84,2.34,2,4,...,2,2,0,1,0,0,0,0,0,2016
6,6,294,37623,TRUE,0.3,2.15,6.55,-1.29,1,2,...,2,2,0,1,0,0,0,0,0,2016
7,7,294,37623,FALSE,0,2.15,6.55,0,0,0,...,0,0,0,0,0,0,0,0,0,2016
8,8,294,37623,TRUE,1.1,2,6.5,-0.05,0,2,...,1,0,0,0,0,0,0,0,0,2016
9,9,294,37623,TRUE,4.4,2.3,7.2,0.7,2,4,...,3,1,0,0,0,0,0,0,0,2016


In [92]:
# Removendo colunas
scouts_raw_2016_ano = scouts_raw_2016_ano.drop('PrecoVariacao', 'Mando', 'Participou', 'Titular', 'Substituido', 'TempoJogado', 'FS', 'PE', 'A', 'FT', 'FD', 'FF', 'G', 'I', 'PP', 'RB', 'FC', 'GC', 'CA', 'CV', 'SG', 'DD', 'DP', 'GS')

In [93]:
# remove todas as linhas com rodada == 0 | scouts_raw_2016_ano = scouts_raw_2016_ano.drop('')
scouts_raw_2016_ano_clean = scouts_raw_2016_ano[scouts_raw_2016_ano['Rodada'] > 0]

In [94]:
scouts_raw_2016_ano_clean.limit(10).toPandas()

,Rodada,ClubeID,AtletaID,Pontos,PontosMedia,Preco,ano
0,1,294,37623,5.3,5.3,6.84,2016
1,2,294,37623,-0.5,2.4,6.67,2016
2,3,294,37623,2.3,2.37,6.98,2016
3,4,294,37623,-2.7,1.1,5.5,2016
4,5,294,37623,8.2,2.52,7.84,2016
5,6,294,37623,0.3,2.15,6.55,2016
6,7,294,37623,0,2.15,6.55,2016
7,8,294,37623,1.1,2,6.5,2016
8,9,294,37623,4.4,2.3,7.2,2016
9,10,294,37623,0,2.3,7.2,2016


In [95]:
scouts_raw_2016_ano_clean.write.parquet('/cartola/clean/scouts_raw/2016')

In [96]:
scouts_raw_2016_parquet = spark.read.parquet('/cartola/clean/scouts_raw/*')

In [97]:
scouts_raw_2016_parquet.schema
scouts_raw_2016_parquet.limit(100).toPandas()

,AtletaID,Rodada,ClubeID,Pontos,PontosMedia,Preco,ano
0,81121,1,262,-2.3,-2.3,0.79,2015
1,81219,1,262,0,0,1,2015
2,88072,1,262,0,0,1,2015
3,89258,1,262,0,0,1,2015
4,91263,1,262,0,0,1,2015
...,...,...,...,...,...,...,...
95,38229,1,264,0,0,13,2015
96,38398,1,264,0,0,15,2015
97,54395,1,264,0,0,15,2015
98,68744,1,264,0.2,0.2,15.19,2015


In [98]:
#deve-se converter valores em BigDecimal?
scouts_raw_2016_parquet = scouts_raw_2016_parquet.withColumn("Rodada", scouts_raw_2016_parquet["Rodada"].cast(IntegerType()))
scouts_raw_2016_parquet = scouts_raw_2016_parquet.withColumn("ClubeID", scouts_raw_2016_parquet["ClubeID"].cast(IntegerType()))
scouts_raw_2016_parquet = scouts_raw_2016_parquet.withColumn("AtletaID", scouts_raw_2016_parquet["AtletaID"].cast(IntegerType()))

In [99]:
scouts_raw_2016_parquet.limit(10).toPandas()

,AtletaID,Rodada,ClubeID,Pontos,PontosMedia,Preco,ano
0,81121,1,262,-2.3,-2.3,0.79,2015
1,81219,1,262,0,0,1,2015
2,88072,1,262,0,0,1,2015
3,89258,1,262,0,0,1,2015
4,91263,1,262,0,0,1,2015
5,74103,1,262,0,0,2,2015
6,78895,1,262,0,0,2,2015
7,88065,1,262,0,0,2,2015
8,91264,1,262,0,0,3,2015
9,78086,1,262,0,0,4,2015


In [100]:
print(scouts_raw_2016_parquet.count())

90942


## Times

In [24]:
times_2016_df = spark.read.csv("/cartola/data/2016/2016_times.csv", header=True)

In [25]:
sorted_times_2016_df = times_2016_df.sort(times_2016_df.ID.asc())

In [26]:
times_2016_ano = sorted_times_2016_df.withColumn('ano', lit(2016))
times_2016_ano.show()

+---+-------------+----------+-------------+----+
| ID|         Nome|Abreviacao|         Slug| ano|
+---+-------------+----------+-------------+----+
|262|     Flamengo|       FLA|     Flamengo|2016|
|263|     Botafogo|       BOT|     Botafogo|2016|
|264|  Corinthians|       COR|  Corinthians|2016|
|266|   Fluminense|       FLU|   Fluminense|2016|
|275|    Palmeiras|       PAL|    Palmeiras|2016|
|276|    São Paulo|       SAO|    Sao-Paulo|2016|
|277|       Santos|       SAN|       Santos|2016|
|282|  Atlético-MG|       CAM|  Atletico-MG|2016|
|283|     Cruzeiro|       CRU|     Cruzeiro|2016|
|284|       Grêmio|       GRE|       Gremio|2016|
|285|Internacional|       INT|Internacional|2016|
|287|      Vitória|       VIT|      Vitoria|2016|
|292|        Sport|       SPT|        Sport|2016|
|293|  Atlético-PR|       CAP|  Atletico-PR|2016|
|294|     Coritiba|       CFC|     Coritiba|2016|
|303|  Ponte Preta|       PON|  Ponte-Preta|2016|
|315|  Chapecoense|       CHA|  Chapecoense|2016|


In [27]:
times_2016_ano.write.parquet('/cartola/clean/times/2016')

In [28]:
times_2016_parquet = spark.read.parquet('/cartola/clean/times/*')

In [29]:
times_2016_parquet.show()
times_2016_parquet.schema

#Alterar ID para int.(está string)

+---+-------------+----------+-------------+----+
| ID|         Nome|Abreviacao|         Slug| ano|
+---+-------------+----------+-------------+----+
|262|     flamengo|       FLA|     flamengo|2014|
|263|     botafogo|       BOT|     botafogo|2014|
|264|  corinthians|       COR|  corinthians|2014|
|265|        bahia|       BAH|        bahia|2014|
|266|   fluminense|       FLU|   fluminense|2014|
|275|    palmeiras|       PAL|    palmeiras|2014|
|276|    são paulo|       SAO|    sao-paulo|2014|
|277|       santos|       SAN|       santos|2014|
|282|  atlético-mg|       CAM|  atletico-mg|2014|
|283|     cruzeiro|       CRU|     cruzeiro|2014|
|284|       grêmio|       GRE|       gremio|2014|
|285|internacional|       INT|internacional|2014|
|287|      vitória|       VIT|      vitoria|2014|
|288|     criciúma|       CRI|     criciuma|2014|
|290|        goiás|       GOI|        goias|2014|
|292|        sport|       SPO|        sport|2014|
|293|  atlético-pr|       CAP|  atletico-pr|2014|


StructType(List(StructField(ID,StringType,true),StructField(Nome,StringType,true),StructField(Abreviacao,StringType,true),StructField(Slug,StringType,true),StructField(ano,IntegerType,true)))

In [30]:
times_2016_parquet = times_2016_parquet.withColumn("ID", times_2016_parquet["ID"].cast(IntegerType()))

In [31]:
times_2016_parquet.schema

StructType(List(StructField(ID,IntegerType,true),StructField(Nome,StringType,true),StructField(Abreviacao,StringType,true),StructField(Slug,StringType,true),StructField(ano,IntegerType,true)))

In [32]:
times_2016_parquet.show()

print(times_2016_parquet.count())

+---+-------------+----------+-------------+----+
| ID|         Nome|Abreviacao|         Slug| ano|
+---+-------------+----------+-------------+----+
|262|     flamengo|       FLA|     flamengo|2014|
|263|     botafogo|       BOT|     botafogo|2014|
|264|  corinthians|       COR|  corinthians|2014|
|265|        bahia|       BAH|        bahia|2014|
|266|   fluminense|       FLU|   fluminense|2014|
|275|    palmeiras|       PAL|    palmeiras|2014|
|276|    são paulo|       SAO|    sao-paulo|2014|
|277|       santos|       SAN|       santos|2014|
|282|  atlético-mg|       CAM|  atletico-mg|2014|
|283|     cruzeiro|       CRU|     cruzeiro|2014|
|284|       grêmio|       GRE|       gremio|2014|
|285|internacional|       INT|internacional|2014|
|287|      vitória|       VIT|      vitoria|2014|
|288|     criciúma|       CRI|     criciuma|2014|
|290|        goiás|       GOI|        goias|2014|
|292|        sport|       SPO|        sport|2014|
|293|  atlético-pr|       CAP|  atletico-pr|2014|


## DEMONSTRAÇÃO

In [ ]:
partidas_2014_df = spark.read.csv("/cartola/data/2014/2014_jogadores.csv", header=True)
partidas_2014_ct = partidas_2014_df.withColumn('time', regexp_replace('home_team', ' - RJ', ''))
final_partidas = partidas_2014_ct.withColumn('time_low', lower(col('time'))).show(truncate=False)

In [ ]:
with_ano_partidas = partidas_2014_ct.withColumn('ano', lit(2014)).show(truncate=False)
with_ano_partidas.show()